In [21]:
import cpp_soft_info

from cpp_soft_info import soft_reweight_pymatching

## Load data

In [22]:
from result_saver import SaverProvider
provider = SaverProvider()

job = provider.retrieve_job("cmyhbrqrmwhg008bs4h0") # Mumbai job
# job = provider.retrieve_job("cn6g47862r90008810pg") # Sherbrooke job

memory = job.result().get_memory() 
print(memory.shape)

(1000, 9)


In [23]:
from soft_info import get_repcode_IQ_map

# Mumbai
layout = [25, 19, 26, 22, 16] # To implement: into scratch job_data
d = 3
synd_rounds = 3

# # Sherbrooke
# layout = get_repcode_layout(30, provider.get_backend('ibm_sherbrooke') )
# synd_rounds = 35

print("Generating the qubit mapping...")
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes

Generating the qubit mapping...


# How to use

In [24]:
from Scratch import create_or_load_kde_grid
from cpp import process_scaler_dict

# Example usage
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-22T10:30:00" # "2023-11-22T" works too 
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, "cmyhbrqrmwhg008bs4h0", 300, 2, other_date=other_date)
# grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, "cn6g47862r90008810pg", 300, 2, other_date=other_date)
print("len of grid_dict (num of qubits):", len(grid_dict.keys()))

counts = cpp_soft_info.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)
# (sorted(counts.items(), key=lambda x: x[1], reverse=True))

len of grid_dict (num of qubits): 27


In [26]:
import numpy as np
import stim
import pymatching

from soft_info import get_repcode_IQ_map

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=synd_rounds,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

print("Generating the qubit mapping...")
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes

Generating the qubit mapping...


# decode_IQ_shots

In [33]:
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory, synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict, 6e-3, 0.49, 0.1)

print("num_errors:", num_errors)

num_errors: 18


# Test

In [14]:
print(len(cpp_soft_info.get_edges(matching._matching_graph)))
cpp_soft_info.processGraph_test(matching._matching_graph)

21
Processing graph


In [15]:
IQ_data = memory[0]
print(IQ_data.shape)
IQ_data = IQ_data.reshape(1, -1)
print(IQ_data.shape)


(9,)
(1, 9)


In [16]:
cpp_soft_info.soft_reweight_pymatching(matching._matching_graph, IQ_data, synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict, 6.836e-3, 0.49, 0.1) 

# Decode

In [34]:
from tqdm import tqdm
from soft_info import counts_to_det_syndr, draw_matching_graph

VERBOSE = False

actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))[:1000]):
    matching = pymatching.Matching.from_detector_error_model(model)
    i += 1
    IQ_data = memory[shot]
    IQ_data_reshaped = IQ_data.reshape(1, -1)

    counts = cpp_soft_info.get_counts(IQ_data_reshaped, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)
    count_key = next(iter(counts.keys()))

    # if count_key == '000 00 00 00':
        #print("Skipping all zeros")
        # continue    

    cpp_soft_info.soft_reweight_pymatching(matching._matching_graph, IQ_data_reshaped, synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict, 6.836e-3, 0.49, 0.1) 


    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    actual_observables = [(int(count_key[0])+0)%2]
    if predicted_observables == actual_observables: #== [0]:
        continue
    num_errors += 1
    # print("num_errors:", num_errors)
    
    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count str to syndromes:", array_processed_string)


    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        print("Actual flip:", actual_observables)
    

    if VERBOSE:
        draw_matching_graph(matching, d, synd_rounds, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(6, 6))



print("Num errors:", num_errors)

100%|██████████| 1000/1000 [00:14<00:00, 69.52it/s]

Num errors: 18
